In [7]:
%pwd


'c:\\Users\\devan\\Downloads\\Brain-Tumor-'

In [6]:
import os
os.chdir("../")

In [13]:
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf
from Brain_tumor_classifier.constants import *
from Brain_tumor_classifier.utils.common import read_yaml, create_directories, save_json

# ---------------- CONFIG CLASS ----------------
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    test_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

# ---------------- CONFIGURATION MANAGER ----------------
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            test_data="artifacts/data_ingestion/brain_tumor_dataset/test",  # full test set
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

# ---------------- EVALUATION CLASS ----------------
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _test_generator(self):
        # No validation split; entire test set
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            shuffle=False,
            class_mode="categorical"
        )

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.test_data,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._test_generator()
        self.score = self.model.evaluate(self.test_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("test_scores.json"), data=scores)

# ---------------- PIPELINE ----------------
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.save_score()
    print(f"Test accuracy: {evaluation.score[1]*100:.2f}%")
except Exception as e:
    raise e


[2025-08-29 00:16:46,628: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-29 00:16:46,630: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-29 00:16:46,633: INFO: common: created directory at: artifacts]
[2025-08-29 00:16:46,783: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 51 images belonging to 2 classes.
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.8039 - loss: 0.3917
[2025-08-29 00:16:50,400: INFO: common: json file saved at: test_scores.json]
Test accuracy: 80.39%
